# Introduction to VQE

Mizore provides a general framework for adaptive circuit constrcution in Variational Quantum Eigensolver (VQE). VQE is a class of quantum algorithms aiming to solve the ground state energy of a Hamiltonian. 
$\newcommand\ket[1]{|#1\rangle} \newcommand\bra[1]{\langle#1|}$

During a VQE run, a parameterized trial wavefunction $\ket{\Psi(\vec{\varphi})}=\hat{U}(\vec{\varphi})\ket{\Psi_0}$ is prepared on quantum computers. And the expectation value $E = \bra{\Psi(\vec{\varphi})}\hat H\ket{\Psi(\vec{\varphi})}$ should be minimized. By the variational law, this minimized expectation value is larger or equal to the ground state energy. If the ansatz $\hat{U}(\vec{\varphi})$ is chosen properly, the outcome of VQE can be very closed to the true ground state energy.

The essential steps of a VQE includes
- Construction of the problem Hamiltonian
    - Active space selection
    - Fermion-qubit transformation (Encoding)
    - Stationary qubit reduction
    - Penalty terms
- Construction of the ansatz
    - Building block
    - Construction method
- Parameter Optimization

For every step of VQE, Mizore provides easy-to-learn and versitile functions for the users. In the following chapter, the usage of the functions will be shown.

## Hamiltonian Construction

In Mizore the problem Hamiltonian can be easily constrcuted by the package `HamiltonianGenerator`

In [1]:
from HamiltonianGenerator.TestHamiltonian import make_example_H2

energy_obj=make_example_H2()

Symmetry: Dooh  is used when build the molecule.


Notice that energy_obj is not a `QubitOperator`. Instead, it is a `EnergyObjective` which includes more information about the task, such as the Hartree-Fock initialization and energy to terminate the calculation.

In [2]:
print("Number of qubits:",energy_obj.n_qubit)
print("Initial block:",energy_obj.init_block)
print("Chemical accuarte energy:",energy_obj.obj_info["terminate_cost"])

Number of qubits: 4
Initial block: Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Chemical accuarte energy: -1.1362838344885025


For advanced instruction of Hamiltonian, please see #TODO

## Circuit (ansatz) constrction

### Block

The class for the building blocks of a quantum circuit is defined in *Blocks* as `Block`. A `Block` is a piece of quantum circuit that have certain physical meaning. Here we introduce some common blocks.

- The High-dimensional rotation block which implements the operation $e^{iPt}$, where $P$ is a pauli word (tensor product of pauli operators on qubits) and $t$ is an adjustable parameter. This block is commonly used for simulation the evolution of a system.

In [3]:
from Blocks import RotationEntangler
print(RotationEntangler((1,2,3),(3,2,1)))

Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX


- The hardware-efficient ansatz which is a layer of full rotation on each qubit, followed by a entangling operation. Here, the entangling operation we choose is CNOT gates between adjacent qubits.

In [4]:
from Blocks import HardwareEfficientEntangler
print(HardwareEfficientEntangler((1,2,3)))

Type:HardwareEfficientEntangler; Para Num:9; Qsubset:(1, 2, 3)


For more instruction of Blocks, please see [Blocks](Blocks.ipynb).

### BlockCircuit

In Mizore, the Blocks are arranged by `BlockCircuit`, which contains a list of Blocks. Convienient gate count is implemented.

In [5]:
from Blocks import BlockCircuit
from Blocks import HardwareEfficientEntangler,RotationEntangler

n_qubit=4
bc=BlockCircuit(n_qubit)
bc.add_block(HardwareEfficientEntangler((0,1)))
bc.add_block(RotationEntangler((1,2,3),(3,2,1)))
bc.add_block(HardwareEfficientEntangler((2,3)))
print(bc)
print("Gate used:",bc.get_gate_used())

{1, 2, 3}
Block Num:3; Qubit Num:4
Block list:
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(2, 1)
Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(2, 3)
Gate used: {'CNOT': 8, 'SingleRotation': 9, 'TimeEvolution': 0}


### Optimizer

#TODO

In [6]:
from ParameterOptimizer import BasinhoppingOptimizer
bc=BlockCircuit(n_qubit)
bc.add_block(energy_obj.init_block) # Add the Hartree-Fock initialization to the circuit
bc.add_block(HardwareEfficientEntangler((0,2)))
print(bc)
bc.set_all_block_active() # Make every block adjustable
optimizer=BasinhoppingOptimizer(random_initial=0.1) # The initial parameter will be a random value between -0.1 and +0.1
energy, amp = optimizer.run_optimization(bc,energy_obj.get_cost())
print("Energy:",energy)
print("Amplitutes", amp)

Block Num:2; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(0, 2)
Energy: -1.1372838344858553
Amplitutes [-5.32835540e-05 -1.56273990e+00 -4.25445061e-02  2.25573186e-01
  5.31861272e-05 -1.57906208e+00]


#TODO

In [7]:
from Blocks._utilities import get_circuit_energy
hamiltonian=energy_obj.hamiltonian
print("Energy before adjust:", get_circuit_energy(bc,hamiltonian))
bc.adjust_parameter_on_active_position(amp) #Update the parameters on the active blocks
print("Energy Now:", get_circuit_energy(bc,hamiltonian))

Energy before adjust: 0.4626181460271928
Energy Now: -1.1372838344858553


Imaginary Time Evolution (ITE) on ansatz can be also easily carried out

In [8]:
from ParameterOptimizer import ImaginaryTimeEvolutionOptimizer
bc=BlockCircuit(n_qubit)
bc.add_block(energy_obj.init_block) # Add the Hartree-Fock initialization to the circuit
bc.add_block(HardwareEfficientEntangler((0,2)))
bc.set_all_block_active() # Make every block adjustable
%matplotlib inline
n_step=10
optimizer=ImaginaryTimeEvolutionOptimizer(random_adjust=0.1,verbose=True, n_step=n_step, stepsize=1e-1, max_increase_n_step=n_step,fig_path=None)
#Change fig_path to "screen" to see the graphs
energy, amp = optimizer.run_optimization(bc,energy_obj.get_cost())

Lowest History Energy: 0.45966358233401916 ; Energy Now: 0.45966358233401916
Lowest History Energy: 0.4587790286611257 ; Energy Now: 0.4587790286611257
Lowest History Energy: 0.457443197408346 ; Energy Now: 0.457443197408346
Lowest History Energy: 0.45412826251556854 ; Energy Now: 0.45412826251556854
Lowest History Energy: 0.45313041455487146 ; Energy Now: 0.45313041455487146
Lowest History Energy: 0.4526682226007636 ; Energy Now: 0.4526682226007636
Lowest History Energy: 0.4505917382006908 ; Energy Now: 0.4505917382006908
Lowest History Energy: 0.44953102557868624 ; Energy Now: 0.44953102557868624
Lowest History Energy: 0.44953102557868624 ; Energy Now: 0.44964103086315044
Lowest History Energy: 0.44953102557868624 ; Energy Now: 0.4496873627468931
